## Minimize walking distance for (non-connecting) departing passengers

$$
\min \sum_{f \in F, g \in G} W_g P_f M_{f, g}
$$

Where: 

- $W_g$ is the walking distance from nearest security checkpoint to the gate \
- $P_{f}$ is the estimated number of departing passengers in flight $f$ \
- $M_{f, g}$ is the binary decision variable. $M_{f, g} = 1$ if flight $f$ is assigned to gate $g$, $0$ otherwise

Constraints:

- No two aircrafts can occupy the same gate at the same time
- If an aircraft is scheduled to depart within two hours of when it arrives, the in-bound and out-bound flights must be assigned the same gate


Assumptions:
- Gates are all shared among airlines.
- Any aircraft can be parked at any gate.
- Walking distances are from nearest security checkpoint to gate.
- Only considering flights from terminals A, B, and C
- Number of passengers are assumed based on the aircraft size: 75, 150, and 300 for regional, narrow-body, and wide-body aircraft, respectively.
- If there's at least two hours between when an aircraft arrives at and departs from DFW, we assume it leaves it's arrival gate after 30 minutes for a hanger and enters its next gate 90 minutes before it's scheduled departure time

### Second objective function also in code below (commented out) for (non-connecting) arriving passengers

$$
\min \sum_{f \in F, g \in G} W_b P_f M_{f, g}
$$

Where: 

- $W_b$ is the walking distance from gate to nearest baggage claim \
- $P_{f}$ is the estimated number of arriving passengers in flight $f$ \
- $M_{f, g}$ is the binary decision variable. $M_{f, g} = 1$ if flight $f$ is assigned to gate $g$, $0$ otherwise

In [ ]:
using JuMP, Gurobi, CSV, DataFrames

# -------------------------------
# Load Data and Setup Parameters
# -------------------------------

file_path = "Data/May_2nd_Formatted.csv"
df = CSV.read(file_path, DataFrame)

walking_distances_file = "Data/Walking Distances Arriving and Departing Pax.csv"
walking_distances = CSV.read(walking_distances_file, DataFrame)

# Separate arriving and departing flights
departing_indices = findall(df.IsDeparting .== "Y")
arriving_indices  = findall(df.IsDeparting .== "N")
F_dep = length(departing_indices)  # Number of departing flights
F_arr = length(arriving_indices)     # Number of arriving flights
F     = nrow(df)                     # Total flights
G     = 96                         # Number of gates

# Define enter and exit gate times
df[!, :EnterGateTime] = df.ArrivalTimeMinutes
df[!, :ExitGateTime]  = df.OffTimeMinutes

BUFFER_TIME = 0   # Buffer time (modifiable parameter)

# Walking times to gates from TSA
W_g = walking_distances.TSA_to_Gate

# Walking times to gates from baggage claim
W_b = walking_distances.Gate_to_Bag

# Passenger count for flight f (if PassengersArr > 0 then use it, else use PassengersDept)
P_f = [ df.PassengersArr[f] > 0 ? df.PassengersArr[f] : df.PassengersDept[f] for f in 1:F ]

# -------------------------------
# Define the Model
# -------------------------------

model = Model(Gurobi.Optimizer)
@variable(model, M[1:F, 1:G], Bin)

# Objective: Minimize total walking distance for DEPARTING passengers
# (Note: the departing flights are referenced via departing_indices)
@objective(model, Min, 
    sum(W_g[g] * P_f[departing_indices[f]] * M[departing_indices[f], g] for f in 1:F_dep, g in 1:G) + sum(W_b[g] * P_f[arriving_indices[f]] * M[arriving_indices[f], g] for f in 1:F_arr, g in 1:G)
)

# Objective: Minimize total walking distance for ARRIVING passengers
# (Note: the arriving flights are referenced via arriving_indices)
# @objective(model, Min, 
#     sum(W_b[g] * P_f[arriving_indices[f]] * M[arriving_indices[f], g] for f in 1:F_arr, g in 1:G)
# )

# Each flight is assigned exactly one gate
@constraint(model, [f in 1:F], sum(M[f, g] for g in 1:G) == 1)

# -------------------------------
# Precompute Conflict Pairs
# -------------------------------

# These are pairs of flights (f1,f2) that overlap in time 
# (with a buffer added to the exit time) and belong to different aircraft.
conflict_pairs = Vector{Tuple{Int, Int}}()
for f1 in 1:(F-1)
    for f2 in (f1+1):F
        if df.TailNumber[f1] != df.TailNumber[f2]
            enter1  = df.EnterGateTime[f1]
            depart1 = df.ExitGateTime[f1] + BUFFER_TIME
            enter2  = df.EnterGateTime[f2]
            depart2 = df.ExitGateTime[f2] + BUFFER_TIME
            if (enter1 < depart2) && (enter2 < depart1)
                push!(conflict_pairs, (f1, f2))
            end
        end
    end
end

# Add constraints: no two conflicting flights may be assigned to the same gate.
for (f1, f2) in conflict_pairs
    for g in 1:G
        @constraint(model, M[f1, g] + M[f2, g] <= 1)
    end
end

# -------------------------------
# Precompute Same‐Gate Pairs for Connections
# -------------------------------

# These are pairs where an arriving flight and a departing flight
# (with the same tail number) must be assigned the same gate 
# if the departing flight’s start time is within 2 hours of the arriving flight’s exit.
same_gate_pairs = Vector{Tuple{Int, Int}}()
for f1 in arriving_indices
    for f2 in departing_indices
        if df.TailNumber[f1] == df.TailNumber[f2] && (df.ExitGateTime[f1] + 120 >= df.EnterGateTime[f2])
            push!(same_gate_pairs, (f1, f2))
        end
    end
end

# Add same‐gate constraints
for (f1, f2) in same_gate_pairs
    for g in 1:G
        @constraint(model, M[f1, g] == M[f2, g])
    end
end

# -------------------------------
# Solve the Model
# -------------------------------

optimize!(model)




## Given optimal gate assignments, the code below computes average walking distance for departing, arriving, and connecting passengers.

In [5]:
# Get the optimal objective value without overwriting the JuMP function name
opt_obj = JuMP.objective_value(model)

# Compute average walking distance for departing passengers:
# Total number of departing passengers is the sum of df.PassengersDep.
total_departing_passengers = sum(df.PassengersDept)
avg_departing_wd = opt_obj / total_departing_passengers

# Determine the assigned gate for each flight from the model’s decision variable M.
# (Assuming M is a F×G matrix with binary values where 1 indicates a flight is assigned to a gate.)
F, G = size(M)
assigned_gate = Vector{Int}(undef, F)
for f in 1:F
    # Find the gate g with assignment value >= 0.5 (i.e. assigned)
    assigned_gate[f] = findfirst(g -> JuMP.value(M[f, g]) >= 0.5, 1:G)
end

# Compute average walking distance for departing passengers:
# For departing flights (where df.IsDeparting == "Y"), use df.PassengersDept and the walking distance
# from the assigned gate to the nearest security checkpoint (from walking_distances.TSA_to_Gate).
departing_indices = findall(x -> x == "Y", df.IsDeparting)
total_departing_passengers = sum(df.PassengersDept[departing_indices])
total_departing_wd = 0.0
for f in departing_indices
    gate = assigned_gate[f]
    # Find the TSA walking distance for the given gate.
    idx = findfirst(==(gate), walking_distances.Gate_Int)
    TSA_distance = walking_distances.TSA_to_Gate[idx]
    total_departing_wd += df.PassengersDept[f] * TSA_distance
end
avg_departing_wd = total_departing_wd / total_departing_passengers

# Compute average walking distance for arriving passengers:
# For arriving flights (where df.IsDeparting == "N"), use df.PassengersArr and the walking distance
# from the assigned gate to the nearest baggage claim (from walking_distances.Gate_to_Baggage).
arrival_indices = findall(x -> x == "N", df.IsDeparting)
total_arriving_passengers = sum(df.PassengersArr[arrival_indices])
total_arriving_wd = 0.0
for f in arrival_indices
    gate = assigned_gate[f]
    # Find the baggage claim walking distance for the given gate.
    idx = findfirst(==(gate), walking_distances.Gate_Int)
    baggage_distance = walking_distances.Gate_to_Bag[idx]
    total_arriving_wd += df.PassengersArr[f] * baggage_distance
end
avg_arriving_wd = total_arriving_wd / total_arriving_passengers

# Compute average walking distance for connecting passengers:
# Use the connections_matrix (100×100) where each (i,j) entry gives the number of passengers connecting
# from flight i to flight j, and use the walking distance from "Walking Distances Gate-to-Gate.csv"
# (stored in walking_distances_gate_to_gate) to get the distance between the gates assigned to flights i and j.
total_connection_wd = 0.0
total_connection_passengers = 0

walking_distance_file = "Data/Walking Distances Gate-to-Gate.csv"
walking_distances = CSV.read(walking_distance_file, DataFrame)
walking_distances_gate_to_gate = CSV.read(walking_distance_file, DataFrame)

connections_matrix_file = "Data/small_connections_matrix.csv"
conn_mat = CSV.read(connections_matrix_file, DataFrame)
conn_mat = Matrix(conn_mat)
for i in 2:size(conn_mat, 1)
    for j in 2:size(conn_mat, 2)
        num_connect = conn_mat[i, j]
        if num_connect > 0
            gate_i = assigned_gate[i]
            gate_j = assigned_gate[j]
            # Obtain the walking distance from gate_i to gate_j.
            gate_to_gate_wd = walking_distances_gate_to_gate[gate_i, gate_j]
            total_connection_wd += num_connect * gate_to_gate_wd
            total_connection_passengers += num_connect
        end
    end
end
avg_connection_wd = total_connection_wd / total_connection_passengers

# Print the computed metrics
println("Optimal Objective Value: ", opt_obj)
println("Average Departing Passenger Walking Distance: ", avg_departing_wd)
println("Average Arriving Passenger Walking Distance: ", avg_arriving_wd)
println("Average Connecting Passenger Walking Distance: ", avg_connection_wd)
println("Total Departing Passengers: ", total_departing_passengers)
println("Total Arriving Passengers: ", total_arriving_passengers)
println("Total Connection Passengers: ", total_connection_passengers)


ArgumentError: ArgumentError: column name :Gate_Int not found in the data frame

## Incorportating arriving and connecting passengers

$$
\min \sum_{f \in F, g \in G} W_g P_{df} M_{f, g} + \lambda * \sum_{f \in F, g \in G} W_b P_{af} M_{f, g} + \alpha * \sum_{f_1, f_2 \in F} T_{f_1, f_2} \cdot \sum_{g_1, g_2 \in G} W_{ g_1, g_2} \cdot  M_{f_1, g_1} \cdot M_{f_2, g_2}
$$

Where: 

$W_g$ is the walking distance from nearest security checkpoint to the gate \
$W_b$ is the walking distance from the gate to the nearest baggage claim \
$P_{df}$ is the estimated number of departing passengers in flight f \
$P_{af}$ is the estimated number of arriving passengers in flight f \
$W_{g_1, g_2}$ is the walking distance between gate $g_1$ and gate $g_2$ \
$T_{f_1, f_2}$ is the number of transfer passengers from flight $f_1$ to flight $f_2$ \
$M_{f, g}$ is the binary decision variable for gate assignments

We linearize $M_{f_1, g_1} \cdot M_{f_2, g_2}$ to make the problem MILP introducing an auxillary variable:

$$
Z_{f_1, g_1, f_2, g_2} \leq M_{f_1, g_1}, \\ Z_{f_1, g_1, f_2, g_2} \leq M_{f_2, g_2} \\
Z_{f_1, g_1, f_2, g_2} \geq M_{f_1, g_1} + M_{f_2, g_2} - 1 \\
Z{f_1, g_1, f_2, g_2} \in \{0,1\}
$$

In [7]:
using JuMP, Gurobi, CSV, DataFrames

# -------------------------------
# Load Data and Setup Parameters
# -------------------------------

file_path = "Data/Small_May_2nd_Formatted.csv"
df = CSV.read(file_path, DataFrame)

walking_distances_file1 = "Data/Walking Distances Arriving and Departing Pax.csv"
walking_distance_file2 = "Data/Walking Distances Gate-to-Gate.csv"
walking_distances = CSV.read(walking_distances_file1, DataFrame)
walking_distances_gate_to_gate = CSV.read(walking_distance_file2, DataFrame)

connections_matrix_file = "Data/small_connections_matrix.csv"
connections_matrix = CSV.read(connections_matrix_file, DataFrame)

# Separate arriving and departing flights
departing_indices = findall(df.IsDeparting .== "Y")
arriving_indices  = findall(df.IsDeparting .== "N")
F_dep = length(departing_indices)   # Number of departing flights
F_arr = length(arriving_indices)    # Number of arriving flights
F     = nrow(df)                    # Total flights
G     = 96                          # Number of gates

# Define enter and exit gate times
df[!, :EnterGateTime] = df.ArrivalTimeMinutes
df[!, :ExitGateTime]  = df.OffTimeMinutes

BUFFER_TIME = 0   # Buffer time (modifiable parameter)

# Passenger counts
P_df = [df.PassengersDept[departing_indices[f]] for f in 1:F_dep]  # Departing pax
P_af = [df.PassengersArr[arriving_indices[f]]  for f in 1:F_arr]   # Arriving pax

# Transfer passengers matrix T_f1_f2(f1,f2)
T_f1_f2 = zeros(F, F)  # Initialize F×F matrix
T_f1_f2[1:size(connections_matrix,1), 1:size(connections_matrix,2)-1] = Matrix(connections_matrix[1:size(connections_matrix,1), 2:end])

# Walking distances
W_g       = walking_distances.TSA_to_Gate     # Gate distance from security
W_b       = walking_distances.Gate_to_Bag     # Gate distance to baggage claim
W_g1_g2   = zeros(G, G)  # Initialize G×G matrix
W_g1_g2[1:95, 1:95] = Matrix(walking_distances_gate_to_gate[1:95, 2:end])  # Fill first 95×95 elements

# Define weights (λ, α) from the objective in the image
lambda_ = 1.0
alpha   = 1.0

# Pre-process connecting flight pairs
connecting_pairs = Vector{Tuple{Int, Int}}()
for f1 in 1:F
    for f2 in 1:F
        if T_f1_f2[f1, f2] > 0
            push!(connecting_pairs, (f1, f2))
        end
    end
end

# -------------------------------
# Define the Model
# -------------------------------
model = Model(Gurobi.Optimizer)

# Gate‐assignment decision variables
@variable(model, M[1:F, 1:G], Bin)

# Auxiliary variables for linearizing M[f1,g1] * M[f2,g2]
# @variable(model, Z[1:F, 1:G, 1:F, 1:G], Bin)
@variable(model, Z[connecting_pairs, 1:G, 1:G], Bin)


# Objective:
#   min ∑( W_g[g]*P_df[f] * M(...) )
#     + λ * ∑( W_b[g]*P_af[f] * M(...) )
#     + α * ∑( T_f1_f2[f1,f2] * W_g1_g2[g1,g2] * Z[f1,g1,f2,g2] )

"""
@objective(model, Min,
    sum(W_g[g] * P_df[f] * M[departing_indices[f], g] for f in 1:F_dep, g in 1:G)
  + lambda_ * sum(W_b[g] * P_af[f] * M[arriving_indices[f], g] for f in 1:F_arr, g in 1:G)
  + alpha   * sum(
        T_f1_f2[f1, f2] * W_g1_g2[g1, g2] * Z[f1, g1, f2, g2]
        for f1 in 1:F, f2 in 1:F, g1 in 1:G, g2 in 1:G
    )
)
    """

@objective(model, Min,
    sum(W_g[g] * P_df[f] * M[departing_indices[f], g] for f in 1:F_dep, g in 1:G)
    + lambda_ * sum(W_b[g] * P_af[f] * M[arriving_indices[f], g] for f in 1:F_arr, g in 1:G)
    + alpha * sum(
        T_f1_f2[f1, f2] * W_g1_g2[g1, g2] * Z[(f1,f2), g1, g2]
        for (f1,f2) in connecting_pairs, g1 in 1:G, g2 in 1:G
    )
)

# Linearization constraints for Z = M[f1,g1] * M[f2,g2]:
"""
@constraints(model, begin
    [f1 in 1:F, g1 in 1:G, f2 in 1:F, g2 in 1:G], Z[f1,g1,f2,g2] <= M[f1,g1]
    [f1 in 1:F, g1 in 1:G, f2 in 1:F, g2 in 1:G], Z[f1,g1,f2,g2] <= M[f2,g2]
    [f1 in 1:F, g1 in 1:G, f2 in 1:F, g2 in 1:G], Z[f1,g1,f2,g2] >= M[f1,g1] + M[f2,g2] - 1
end)
"""
# Modified linearization constraints
@constraints(model, begin
    [(f1,f2) in connecting_pairs, g1 in 1:G, g2 in 1:G], 
        Z[(f1,f2),g1,g2] <= M[f1,g1]
    [(f1,f2) in connecting_pairs, g1 in 1:G, g2 in 1:G], 
        Z[(f1,f2),g1,g2] <= M[f2,g2]
    [(f1,f2) in connecting_pairs, g1 in 1:G, g2 in 1:G], 
        Z[(f1,f2),g1,g2] >= M[f1,g1] + M[f2,g2] - 1
end)

# Each flight assigned to exactly one gate
@constraint(model, [f in 1:F], sum(M[f, g] for g in 1:G) == 1)

# -------------------------------
# Precompute Conflict Pairs
# -------------------------------

conflict_pairs = Vector{Tuple{Int, Int}}()
for f1 in 1:(F-1)
    for f2 in (f1+1):F
        if df.TailNumber[f1] != df.TailNumber[f2]
            enter1  = df.EnterGateTime[f1]
            depart1 = df.ExitGateTime[f1] + BUFFER_TIME
            enter2  = df.EnterGateTime[f2]
            depart2 = df.ExitGateTime[f2] + BUFFER_TIME
            if (enter1 < depart2) && (enter2 < depart1)
                push!(conflict_pairs, (f1, f2))
            end
        end
    end
end

# No two conflicting flights may share the same gate
for (f1, f2) in conflict_pairs
    for g in 1:G
        @constraint(model, M[f1, g] + M[f2, g] <= 1)
    end
end

# -------------------------------
# Precompute Same‐Gate Pairs
# -------------------------------
same_gate_pairs = Vector{Tuple{Int, Int}}()
for f1 in arriving_indices
    for f2 in departing_indices
        if df.TailNumber[f1] == df.TailNumber[f2] &&
           (df.ExitGateTime[f1] + 120 >= df.EnterGateTime[f2])
            push!(same_gate_pairs, (f1, f2))
        end
    end
end

# If same tail number and close arrival/departure, force same gate
for (f1, f2) in same_gate_pairs
    for g in 1:G
        @constraint(model, M[f1, g] == M[f2, g])
    end
end

# -------------------------------
# Solve the Model
# -------------------------------
optimize!(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1250U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 530788 rows, 83328 columns and 1144704 nonzeros
Model fingerprint: 0x0021b0ea
Variable types: 0 continuous, 83328 integer (83328 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 308188 rows and 2688 columns (presolve time = 5s) ...
Presolve removed 309436 rows and 3072 columns (presolve time = 10s) ...
Presolve removed 451452 rows and 3072 columns
Presolve time: 12.24s
Presolved: 79336 rows, 80256 columns, 269120 nonzeros
Variable types: 0 continuous, 80256 integer (80256 binary)
Deterministic concurren

In [8]:
# Extract results
assignments = Dict(f => g for f in 1:F, g in 1:G if value(M[f, g]) ≈ 1)

# Create new columns for optimized gate assignments
df[!, :OptDepGate] = Vector{Union{String, Missing}}(missing, nrow(df))
df[!, :OptArrGate] = Vector{Union{String, Missing}}(missing, nrow(df))

# Gate mapping dictionary
gate_mapping = Dict(
    1 => "A8", 2 => "A9", 3 => "A10", 4 => "A11", 5 => "A13", 
    6 => "A14", 7 => "A15", 8 => "A16", 9 => "A17", 10 => "A18", 
    11 => "A19", 12 => "A20", 13 => "A21", 14 => "A22", 15 => "A23", 
    16 => "A24", 17 => "A25", 18 => "A28", 19 => "A29", 20 => "A33", 
    21 => "A34", 22 => "A35", 23 => "A36", 24 => "A37", 25 => "A38", 
    26 => "A39", 27 => "B1", 28 => "B2", 29 => "B3", 30 => "B4", 
    31 => "B5", 32 => "B6", 33 => "B7", 34 => "B9", 35 => "B10", 
    36 => "B11", 37 => "B12", 38 => "B14", 39 => "B16", 40 => "B17", 
    41 => "B18", 42 => "B19", 43 => "B21", 44 => "B22", 45 => "B24", 
    46 => "B25", 47 => "B26", 48 => "B27", 49 => "B28", 50 => "B29", 
    51 => "B30", 52 => "B31", 53 => "B32", 54 => "B33", 55 => "B34", 
    56 => "B35", 57 => "B36", 58 => "B37", 59 => "B38", 60 => "B39", 
    61 => "B40", 62 => "B42", 63 => "B43", 64 => "B44", 65 => "B46", 
    66 => "B47", 67 => "B48", 68 => "B49", 69 => "C2", 70 => "C4", 
    71 => "C6", 72 => "C7", 73 => "C8", 74 => "C10", 75 => "C11", 
    76 => "C12", 77 => "C14", 78 => "C15", 79 => "C16", 80 => "C17", 
    81 => "C19", 82 => "C20", 83 => "C21", 84 => "C22", 85 => "C24", 
    86 => "C26", 87 => "C27", 88 => "C28", 89 => "C29", 90 => "C30", 
    91 => "C31", 92 => "C33", 93 => "C35", 94 => "C36", 95 => "C37", 
    96 => "C39"
)

# Assign gates
for f in 1:F
    gate_number = get(assignments, f, missing)
    if !ismissing(gate_number)
        gate_code = get(gate_mapping, gate_number, missing)
        if df.IsDeparting[f] == "Y"
            df[f, :OptDepGate] = gate_code
        else
            df[f, :OptArrGate] = gate_code
        end
    end
end

println(df)

# Save results
CSV.write("Optimized_Gate_Assignments_Sample_Day.csv", df)

100×31 DataFrame
 Row │ FlightNumber  IsDeparting  Destination  Origin   ArrivalTime  OffTime  ArrivalTimeMinutes  OffTimeMinutes  ArrivalTimeActual  OffTimeActual  ArrivalTimeActualMinutes  OffTimeActualMinutes  Aircraft   FlightType  PassengersArr  PassengersDept  DepGate   ArrGate   SchedDepLocal        SchedArrLocal        OutGateLocal         InGateLocal          SchedDepUtc       SchedArrUtc       OutGateUtc        InGateUtc         TailNumber  EnterGateTime  ExitGateTime  OptDepGate  OptArrGate 
     │ String7       String1      String3      String3  Int64        Int64    Int64               Int64           Int64              Int64          Float64                   Float64               Float64?   String15?   Int64          Int64           String3?  String3?  String31             String31             String31             String31             String31          String31          String31          String31          String7     Int64          Int64         String?     String?    
─

"Optimized_Gate_Assignments_Sample_Day.csv"